## Reproduction of Knowledge Source Integration on Medical Code Prediction

This bonus Jupyter Notebook was created solely by Robert Zhang (ruihaoz2@illinois.edu), my teammate Hannah Ke agreed to abandon this bonus task.

### 1. Reproducibility Summary

The original paper proposed a framework called Knowledge Source Integration (KSI) for improving the automatic assignment of ICD-9 diagnosis codes to clinical notes. The KSI framework incorporated external knowledge sources, such as Wikipedia, into the training process of neural network models. The reproducibility project aimed to replicate the original study's findings and evaluate the performance of the KSI framework using the MIMIC-III dataset.

The report described the methodology used to replicate the original study, including data preprocessing, model descriptions, and hyperparameter tuning. The authors successfully reproduced the KSI model and demonstrated its effectiveness in improving code prediction accuracy. They compared the performance of KSI models with CNN and LSTM to their respective baseline models without external knowledge integration. The results showed that the KSI models outperformed the baseline models, achieving a 3% increase in accuracy.

The report also discussed the challenges faced during reproduction, such as data preprocessing complexities and the difficulty of reproducing the model structure. Recommendations for improving reproducibility included documenting the random seed initialization, providing detailed documentation and instructions, and parameterizing hyperparameters for easier experimentation.

In addition, the authors of the reproducibility report communicated with the original authors to clarify certain aspects of the study and ensure a transparent exchange of information. Overall, the report confirms the reproducibility of the original study's findings and highlights the effectiveness of the KSI framework in improving medical code prediction.

### 2. Data Overview
We have three datasets (two main data sources) for the model:
* NOTEEVENTS.csv (A clinical notes dataset from the MIMIC-III Critical Care Database)
* DIAGNOSES_ICD.csv (The ICD-9 diagnosis codes dataset from the MIMIC-III Database)
* wikipedia_knowledge.txt (The external ICD-9 diagnosis codes knowledge dataset from Wikipedia.)

In [51]:
import pandas as pd
df_NoteEvents = pd.read_csv("NOTEEVENTS_Partial.csv")
df_DiagnosesICD = pd.read_csv("DIAGNOSES_ICD.csv")
with open("wikipedia_knowledge.txt", 'r', encoding='utf-8') as file:
    lines = file.readlines()

Here is the overview of the NOTEEVENTS.csv

In [52]:
df_NoteEvents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,8/4/2151,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,6/14/2118,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,5/25/2119,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,8/18/2124,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,3/25/2162,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


Here is the overview of the DiagnosesICD.csv

In [53]:
df_DiagnosesICD.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


Here is the overview of the wikipedia_knowledge.txt

In [54]:
for line in lines[:5]:
    print(line)

XXXdiseaseXXX   breast cancer  d_174  d_175

Breast cancer



Breast cancer is cancer that develops from breast tissue. Signs of breast cancer may include a lump in the breast, a change in breast shape, dimpling of the skin, fluid coming from the nipple, a newly inverted nipple, or a red or scaly patch of skin. In those with distant spread of the disease, there may be bone pain, swollen lymph nodes, shortness of breath, or yellow skin.

Risk factors for developing breast cancer include being female, obesity, lack of physical exercise, drinking alcohol, hormone replacement therapy during menopause, ionizing radiation, early age at first menstruation, having children late or not at all, older age, prior history of breast cancer, and family history. About 5–10% of cases are due to genes inherited from a person's parents, including BRCA1 and BRCA2 among others. Breast cancer most commonly develops in cells from the lining of milk ducts and the lobules that supply the ducts with milk. Cance

On average, each visit has 11 medical codes. The code vocabulary contains 942 codes. Of those codes, we selected a subset of 344 codes for which we found the corresponding Wikipedia document and used those codes in our experiments. For Wikipedia knowledge dataset, we found 344 of them in MIMIC-III medical code vocabulary. 

### 3. Methodology

In this part, We take one of our model: CNN+KSI as an example. We first load the data, preprocess it, and create batches. Then, we define a CNN model and train it using the loaded data. Finally, we test the model's performance on unseen data.

#### 3.1 Data Preprocessing
The dataset is preprocessed and converted into batches suitable for training and evaluation. This includes:
* Converting labels into one-hot vectors
* Sorting the data by length
* Creating batches with fixed batch size and padding

In [55]:
def func_data_preprocess(data, batch_size=32):
    processed_data = []

    # convert labels to vectors
    for i, note, j in data:
        temp_label = np.zeros(len(lab2ind), dtype=float)
        temp_label[[lab2ind[jj] for jj in j if jj in wikivoc]] = 1.0
        processed_data.append((i, note, temp_label))

    processed_data = np.array(processed_data, dtype=object)

    # sort by length of i
    lengths = np.array([len(item[0]) for item in processed_data])
    sorted_indices = lengths.argsort()
    processed_data = processed_data[sorted_indices]

    # create batches
    batched_data = []
    for start_ix in range(0, len(processed_data) - batch_size + 1, batch_size):
        block = processed_data[start_ix:start_ix+batch_size]
        max_word_num = max(len(item[0]) for item in block)

        # create main matrix
        main_matrix = np.zeros((len(block), max_word_num), dtype=np.int64)
        for i, item in enumerate(block):
            ix = [wor2ind[word] for word in item[0][:max_word_num] if word in wor2ind]
            main_matrix[i, :len(ix)] = ix

        # gather notes and labels
        notes = np.array([item[1] for item in block])
        labels = np.array([item[2] for item in block])

        # wrap in torch variables
        batched_data.append((
            autograd.Variable(torch.from_numpy(main_matrix)),
            autograd.Variable(torch.FloatTensor(notes)),
            autograd.Variable(torch.FloatTensor(labels))
        ))

    return batched_data

#### 3.2 Model Architecture
The CNN model is designed with the following layers:
* An embedding layer with dropout
* Convolutional layers with varying kernel sizes
* Linear layers for similarity learning
* An output layer to generate tag scores

In [56]:
class CNN(nn.Module):
    def __init__(self, batch_size, vocab_size, target_size):
        super(CNN, self).__init__()
        
        # Embedding layer with dropout
        self.embeddings = nn.Sequential(
            nn.Embedding(vocab_size + 1, embed_size, padding_idx=0),
            nn.Dropout(p=0.2)
        )
        
        # Convolutional layers with varying kernel sizes
        self.convs1 = nn.Conv1d(embed_size, conv_out, kernel_size[0])
        self.convs2 = nn.Conv1d(embed_size, conv_out, kernel_size[1])
        self.convs3 = nn.Conv1d(embed_size, conv_out, kernel_size[2])
        
        # Linear layers
        self.hidden2tag = nn.Linear(300, target_size)
        self.layer2 = nn.Linear(embed_size, 1, bias=False)
        self.embedding = nn.Linear(rvocsize, embed_size)
        self.attention = nn.Linear(embed_size, embed_size)
        
        # Activation function
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, vec1, nvec, wiki, simlearning):
        # Apply embedding and dropout to the input
        x = self.embeddings(vec1).transpose(1, 2)

        # Apply convolutional layers
        output1 = self.tanh(self.convs1(x))
        output1 = nn.MaxPool1d(output1.size()[2])(output1)

        output2 = self.tanh(self.convs2(x))
        output2 = nn.MaxPool1d(output2.size()[2])(output2)

        output3 = self.tanh(self.convs3(x))
        output3 = nn.MaxPool1d(output3.size()[2])(output3)

        # Concatenating convolutional outputs
        output4 = torch.cat([output1, output2, output3], 1).squeeze(2)

        # Similarity learning block
        if simlearning:
            # Expand nvec and wiki to match dimensions
            nvec = nvec.view(batch_size, 1, -1).expand(batch_size, wiki.size()[0], -1)
            wiki = wiki.view(1, wiki.size()[0], -1).expand(nvec.size())
            
            # Apply linear layers and activation function
            new = self.embedding(wiki * nvec)
            attention = self.sigmoid(self.attention(new))
            vec3 = self.layer2(new * attention).view(batch_size, -1)
            
            # Compute final tag scores with similarity learning
            tag_scores = self.sigmoid((self.hidden2tag(output4).detach() + vec3))
        else:
            # Compute final tag scores without similarity learning
            tag_scores = self.sigmoid(self.hidden2tag(output4))

        return tag_scores

#### 3.3 Training the Model
Two models are trained in this notebook: 1. Base CNN model 2. CNN model with Knowledge-guided Similarity learning (KSI). For each model, the training process consists of:
* Forward pass through the model to generate tag scores
* Computing the loss between the generated tag scores and the ground truth
* Backward pass to update the model's parameters using the optimizer
* The training is done with early stopping to prevent overfitting.

In [57]:
def func_train_model(model, sim):
    print('start_training')
    modelsaved = []
    modelperform = []
    top = 10

    bestresults = -1
    bestiter = -1
    for epoch in range(5000):
        model.train()
        lossestrain = []

        # Training loop
        for mysentence in train_data_batch:
            model.zero_grad()
            targets = mysentence[2].cuda()
            tag_scores = model(mysentence[0].cuda(), mysentence[1].cuda(), wikivec.cuda(), sim)
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            lossestrain.append(loss.data.mean())

        print(epoch)
        modelsaved.append(copy.deepcopy(model.state_dict()))
        print("--------------------------")
        model.eval()

        recall = []
        
        # Validation loop
        for inputs in val_data_batch:
            targets = inputs[2].cuda()
            tag_scores = model(inputs[0].cuda(), inputs[1].cuda(), wikivec.cuda(), sim)
            loss = loss_function(tag_scores, targets)

            targets = targets.data.cpu().numpy()
            tag_scores = tag_scores.data.cpu().numpy()

            # Compute recall for each validation example
            for iii in range(len(tag_scores)):
                temp = {iiii: tag_scores[iii][iiii] for iiii in range(len(tag_scores[iii]))}
                temp1 = sorted(temp.items(), key=lambda x: x[1], reverse=True)
                thistop = int(np.sum(targets[iii]))
                hit = sum([1.0 for ii in temp1[:max(thistop, top)] if targets[iii][ii[0]] == 1.0])

                if thistop != 0:
                    recall.append(hit / thistop)

        # Calculate and print validation recall
        avg_recall = np.mean(recall)
        print(f'validation top-{top} {avg_recall}')

        # Update model performance
        modelperform.append(avg_recall)
        if avg_recall > bestresults:
            bestresults = avg_recall
            bestiter = len(modelperform) - 1

        # Early stopping
        if (len(modelperform) - bestiter) > 5:
            print(modelperform, bestiter)
            return modelsaved[bestiter]

#### 3.4 Testing the Model
The performance of the trained models is evaluated on the test dataset using various metrics, including recall, AUC-ROC, and F1 score.

In [58]:
def func_test_model(modelstate, sim):
    model = CNN(batch_size, len(wor2ind), len(lab2ind))
    model.cuda()
    loss_function = nn.BCELoss()
    model.eval()

    # Initialize metrics
    recall = []
    lossestest = []
    y_true = []
    y_scores = []

    # Test loop
    for inputs in test_data_batch:
        targets = inputs[2].cuda()
        tag_scores = model(inputs[0].cuda(), inputs[1].cuda(), wikivec.cuda(), sim)
        loss = loss_function(tag_scores, targets)

        targets = targets.data.cpu().numpy()
        tag_scores = tag_scores.data.cpu().numpy()

        lossestest.append(loss.data.mean())
        y_true.append(targets)
        y_scores.append(tag_scores)

        # Compute recall for each test example
        for iii in range(len(tag_scores)):
            temp = {iiii: tag_scores[iii][iiii] for iiii in range(len(tag_scores[iii]))}
            temp1 = sorted(temp.items(), key=lambda x: x[1], reverse=True)
            thistop = int(np.sum(targets[iii]))
            hit = sum([1.0 for ii in temp1[:max(thistop, top)] if targets[iii][ii[0]] == 1.0])

            if thistop != 0:
                recall.append(hit / thistop)

    # Prepare data for metric calculation
    y_true = np.concatenate(y_true, axis=0).T
    y_scores = np.concatenate(y_scores, axis=0).T
    y_true, y_scores = filter_empty_columns(y_true, y_scores)
    y_pred = (y_scores > 0.5).astype(np.int64)

    # Calculate and print metrics
    print('test loss', np.mean([loss.cpu().item() for loss in lossestest]))
    print(f'top-{top}', np.mean(recall))
    print('macro AUC', roc_auc_score(y_true, y_scores, average='macro'))
    print('micro AUC', roc_auc_score(y_true, y_scores, average='micro'))
    print('macro F1', f1_score(y_true, y_pred, average='macro'))
    print('micro F1', f1_score(y_true, y_pred, average='micro'))

#### 3.5 Results
After running the code, you will see the performance metrics for both the base CNN model and the CNN + KSI model. This will give you an idea of the improvements gained by incorporating similarity learning into the CNN architecture.

In [59]:
# train_data_batch=func_data_preprocess(train_data)
# test_data_batch=func_data_preprocess(test_data)
# val_data_batch=func_data_preprocess(val_data)

# # Initialize and train the base model
# model = CNN(batch_size, len(wor2ind), len(lab2ind))
# model.cuda()
# loss_function = nn.BCELoss()
# optimizer = optim.Adam(model.parameters())
# basemodel = func_train_model(model, 0)
# torch.save(basemodel, 'CNN_model')
# print("--------------------------")
# print('CNN alone:')
# func_test_model(basemodel, 0)
# print('===============================')

# # Initialize and train the KSI model
# model = CNN(batch_size, len(wor2ind), len(lab2ind))
# model.cuda()
# model.load_state_dict(basemodel)
# loss_function = nn.BCELoss()
# optimizer = optim.Adam(model.parameters())
# KSImodel = func_train_model(model, 1)
# torch.save(KSImodel, 'KSI_CNN_model')
# print("--------------------------")
# print('KSI+CNN:')
# func_test_model(KSImodel, 1)

### 4. Key Results

This project successful reproduce the original KSI model and prove its ability to significantly improve the automatic assignment of ICD-9 diagnosis codes to clinical notes. The KSI framework was modified to reduce running time, and it was found to outperform models that do not use external knowledge sources.

Two scenarios were tested: KSI + CNN and KSI + LSTM. In both cases, incorporating KSI with the baseline model (CNN or LSTM) resulted in a 3% increase in overall accuracy. This demonstrates the potential of leveraging external sources to improve the accuracy of these models.

In [60]:
table_data = {
    "Evaluation": ["Test loss", "Top 10 acc", "Macro AUC", "Micro AUC", "Macro F1", "Micro F1"],
    "CNN Alone": [0.0387, 0.7553, 0.8323, 0.9675, 0.2199, 0.6289],
    "CNN + KSI": [0.0366, 0.7801, 0.8659, 0.9737, 0.2447, 0.6445],
    "LSTM Alone": [0.0341, 0.7617, 0.8360, 0.9687, 0.1996, 0.6423],
    "LSTM + KSI": [0.0318, 0.7901, 0.8710, 0.9758, 0.2471, 0.6552],
}
df_Results = pd.DataFrame(table_data)
df_Results

,Evaluation,CNN Alone,CNN + KSI,LSTM Alone,LSTM + KSI
0,Test loss,0.0387,0.0366,0.0341,0.0318
1,Top 10 acc,0.7553,0.7801,0.7617,0.7901
2,Macro AUC,0.8323,0.8659,0.8360,0.8710
3,Micro AUC,0.9675,0.9737,0.9687,0.9758
4,Macro F1,0.2199,0.2447,0.1996,0.2471
5,Micro F1,0.6289,0.6445,0.6423,0.6552


Hyperparameter tuning was also performed, testing variations in embedding size, hidden dimension, and other parameters. It was concluded that the original hyperparameter settings were already optimized, and changing them did not yield better results.

In [61]:
data = {
    "Hyperparameters": ["Test loss", "Top 10 acc", "Macro AUC", "Micro AUC", "Macro F1", "Micro F1"],
    "Small": [0.7330, 0.0160, 0.5037, 0.4266, 0.0142, 0.0297],
    "Origin": [0.0366, 0.7801, 0.8659, 0.9737, 0.2447, 0.6445],
    "Large": [0.7424, 0.0525, 0.4958, 0.5798, 0.0205, 0.0430]
}
df_hyperparameters = pd.DataFrame(data)
df_hyperparameters

,Hyperparameters,Small,Origin,Large
0,Test loss,0.7330,0.0366,0.7424
1,Top 10 acc,0.0160,0.7801,0.0525
2,Macro AUC,0.5037,0.8659,0.4958
3,Micro AUC,0.4266,0.9737,0.5798
4,Macro F1,0.0142,0.2447,0.0205
5,Micro F1,0.0297,0.6445,0.0430


### 5. Reference

1. Bai, T., & Vucetic, S. (2019). Improving Medical Code Prediction from Clinical Text via Incorporating Online Knowledge Sources. In *The World Wide Web Conference (WWW '19)*. Association for Computing Machinery, New York, NY, USA, 72–82. https://doi.org/10.1145/3308558.3313485

2. Avati, A., Jung, K., Harman, S., Downing, L., Ng, A., & Shah, N. H. (2017). Improving palliative care with deep learning. In *Bioinformatics and Biomedicine (BIBM), 2017 IEEE International Conference on*. IEEE, 311–316.

3. Bahdanau, D., Cho, K., & Bengio, Y. (2014). Neural machine translation by jointly learning to align and translate. *arXiv preprint arXiv:1409.0473 (2014)*.

4. Bai, T., Chanda, A. K., Egleston, B. L., & Vucetic, S. (2017). Joint learning of representations of medical concepts and words from ehr data. In *2017 IEEE International Conference on Bioinformatics and Biomedicine (BIBM)*. IEEE, 764–769.

5. Bai, T., Zhang, S., Egleston, B. L., & Vucetic, S. (2018). Interpretable Representation Learning for Healthcare via Capturing Disease Progression through Time. In *Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining*. ACM, 43–51.

6. Baumel, T., Nassour-Kassis, J., Cohen, R., Elhadad, M., & Elhadad, N. (2017). Multi-label classification of patient notes a case study on ICD code assignment. *arXiv preprint arXiv:1709.09587* (2017).

7. Choi, E., Bahadori, M. T., Schuetz, A., Stewart, W. F., & Sun, J. (2016). Doctor ai: Predicting clinical events via recurrent neural networks. In *Machine Learning for Healthcare Conference*. 301–318.
